In [2]:
%%time

from datetime import datetime, timedelta, timezone
import json
import os
import re
import boto3
from time import sleep
from threading import Thread

import pandas as pd

from sagemaker import get_execution_role, session, Session, image_uris
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.processing import ProcessingJob
from sagemaker.serializers import CSVSerializer

from sagemaker.model import Model
from sagemaker.model_monitor import DataCaptureConfig

session = Session()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
CPU times: user 1.34 s, sys: 296 ms, total: 1.64 s
Wall time: 3.03 s


In [4]:
# Get Execution role
role = get_execution_role()
print("RoleArn:", role)

region = session.boto_region_name
print("Region:", region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
RoleArn: arn:aws:iam::911199926915:role/LabRole
Region: us-east-1


In [5]:
# Setup S3 bucket
# You can use a different bucket, but make sure the role you chose for this notebook
# has the s3:PutObject permissions. This is the bucket into which the data is captured
bucket = session.default_bucket()
print("Bucket:", bucket)
prefix = "Smart_Grid/model_depolyment"

##S3 prefixes
data_capture_prefix = f"{prefix}/datacapture"
s3_capture_upload_path = f"s3://{bucket}/{data_capture_prefix}"

ground_truth_upload_path = (
    f"s3://{bucket}/{prefix}/ground_truth_data/{datetime.now():%Y-%m-%d-%H-%M-%S}"
)

reports_prefix = f"{prefix}/reports"
s3_report_path = f"s3://{bucket}/{reports_prefix}"

##Get the model monitor image
monitor_image_uri = image_uris.retrieve(framework="model-monitor", region=region)

print("Image URI:", monitor_image_uri)
print(f"Capture path: {s3_capture_upload_path}")
print(f"Ground truth path: {ground_truth_upload_path}")
print(f"Report path: {s3_report_path}")

Bucket: sagemaker-us-east-1-911199926915
Image URI: 156813124566.dkr.ecr.us-east-1.amazonaws.com/sagemaker-model-monitor-analyzer
Capture path: s3://sagemaker-us-east-1-911199926915/Smart_Grid/model_depolyment/datacapture
Ground truth path: s3://sagemaker-us-east-1-911199926915/Smart_Grid/model_depolyment/ground_truth_data/2024-02-20-04-04-57
Report path: s3://sagemaker-us-east-1-911199926915/Smart_Grid/model_depolyment/reports


In [6]:
train_model_id, train_model_version, train_scope = "lightgbm-regression-model", "*", "training"
inference_instance_type = "ml.m5.large"
# Retrieve the inference docker container uri
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=train_model_id,
    model_version=train_model_version,
    instance_type=inference_instance_type,
)

Using model 'lightgbm-regression-model' with wildcard version identifier '*'. You can pin to version '2.1.0' for more stable results. Note that models may have different input/output signatures after a major version upgrade.


In [7]:
model_name = f"smart-grid-LGBM-pred-model-monitor-{datetime.utcnow():%Y-%m-%d-%H%M}"

model_url = "s3://sagemaker-us-east-1-911199926915/Smart-Grid-prediction-lightGBM/output/LGBM-2024-02-04-19-49-05/SmartGrid-lightgbm-r-240204-1951-003-2a579e3d/output/model.tar.gz"

model = Model(image_uri=deploy_image_uri, model_data=model_url, role=role, sagemaker_session=session)

In [8]:
endpoint_name = f"smart-grid-LGBM-pred-model-quality-monitor-{datetime.utcnow():%Y-%m-%d-%H%M}"
print("EndpointName =", endpoint_name)
JsonContentTypes = "application/json"
data_capture_config = DataCaptureConfig(
    enable_capture=True, sampling_percentage=100, destination_s3_uri=s3_capture_upload_path
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    entry_point="inference.py",
    endpoint_name=endpoint_name,
    data_capture_config=data_capture_config,
)

EndpointName = smart-grid-LGBM-pred-model-quality-monitor-2024-02-20-0413
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
------!

In [10]:
from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
predictor = Predictor(
    endpoint_name=endpoint_name, sagemaker_session=session, serializer=JSONSerializer()
)

In [11]:
validate_dataset = "validation_with_predictions.csv"

In [12]:
limit = 200  # Need at least 200 samples to compute standard deviations
i = 0
import json
import time

# Change 'validate_dataset' to your desired output file name
validate_dataset = "output_baseline.csv"

with open(f"test_data/{validate_dataset}", "w") as baseline_file:
    baseline_file.write("prediction,label\n")  # Header

    # Open and read the JSON Lines file
    with open("test_data/validation_data.jsonl", "r") as f:
        for line in f:
            # Parse the JSON object in each line
            data = json.loads(line)
            label = data["label"]
            features = data["features"]

            # Convert features to a JSON string for prediction
            input_json = json.dumps(features)

            # Make the prediction
            prediction_result = predictor.predict(input_json)
            # Assuming the result is a JSON string with a 'predictions' key
            prediction = json.loads(prediction_result)['predictions'][0]

            # Write the prediction and label to the output file
            baseline_file.write(f"{prediction},{label}\n")

            i += 1
            if i > limit:
                break
            print(".", end="", flush=True)
            time.sleep(0.5)

print()
print("Done!")

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from primary with message "{"error": "unsupported content type application/json"}
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/sagemaker_inference/transformer.py", line 142, in transform
    result = self._run_handler_function(
  File "/opt/conda/lib/python3.10/site-packages/sagemaker_inference/transformer.py", line 272, in _run_handler_function
    result = func(*argv)
  File "/opt/ml/model/code/inference.py", line 65, in transform_fn
    raise ValueError('{{"error": "unsupported content type {}"}}'.format(content_type or "unknown"))
ValueError: {"error": "unsupported content type application/json"}
". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/smart-grid-LGBM-pred-model-quality-monitor-2024-02-20-0413 in account 911199926915 for more information.

In [ ]:
limit = 200  # Need at least 200 samples to compute standard deviations
i = 0
import json
with open(f"test_data/{validate_dataset}", "w") as baseline_file:
    baseline_file.write("prediction,label\n")  # our header
    with open("test_data/validation_data.csv", "r") as f:
        for row in f:
            (label, input_cols) = row.split(",", 1)
            prediction = float(predictor.predict(input_cols))
            #data = json.loads(prediction)
            #prediction_number = data['prediction'][0]
            baseline_file.write(f"{prediction},{label}\n")
            i += 1
            if i > limit:
                break
            print(".", end="", flush=True)
            sleep(0.5)
print()
print("Done!")

In [13]:
predictor.delete_model()
predictor.delete_endpoint()